# Activity 5: Minimization Problems and Least Squares

In [1]:
import numpy as np
from numpy import linalg as LA

## Exercise 1: Some Utility Functions
### Positive Definite Matrices
Recall that a symmetric $n\times n$ matrix $K$ is positive semi-definite if $\vec x^T K \vec x\geq 0$ for all $\vec x\in \mathbb R ^ n$, and positive definite if $\vec x^T K \vec x> 0$ for all $\vec 0 \neq\vec x\in \mathbb R ^ n$.
A very helpful characterization of positive definite matrices comes from the following theorem in Olver-Shakiban:

**Theorem 3.43.** A symmetric matrix is positive definite if and only if it is regular and has all positive pivots.

**Exercise: (a)** Below, give a function `is_pos_def` which determines if a matrix `A` is positive definite. We'll do this by checking that:
- `A` is symmetric.
- `A` is regular
- `A` has all positive pivots
Two of these properties are relatively unstable, so we will work up to a specified tolerance, set by default to (say) `10**(-10)`. That is, we'll say `A` is symmetric up to `tolerance` if `abs(A[i,j]-A[j,i])<tolerance`, and that it has positive pivots if each pivot is greater than the supplied tolerance.


<details>
    <summary> 
        <b> Hint:</b> (click to expand)
    </summary>

    A few built-in functions may be helpful
    - `np.any` takes an array of booleans as input and returns `True` if at least one entry is `True`
    - `np.any` takes an array of booleans as input and returns `True` if every entry is `True`
    - `np.array_equal` checks if two arrays are equal on the nose.
</details>


In [2]:
def perm_matrix(n,i,j):
    E=np.identity(n)
    E[[i,j]]=E[[j,i]]
    return E
def elem_matrix(n,i,j,a):
    #constructs the elementary nxn matrix with value a in entry (i,j)
    E=np.identity(n)
    E[i][j]=a
    return E
def swapper(P,L,U,i,k): #OPTIONAL (if you wish to follow the outline)
    n=P.shape[0]
    perm=perm_matrix(n,i,k)
    P=np.dot(perm,P)
    U=np.dot(perm,U)
    L=np.dot(perm,L-np.identity(n))+np.identity(n)
    return (P,L,U)
def my_PLU(A):
    n=A.shape[0] #get number of rows
    U=A.copy() #initialize U
    L=np.identity(n) #initialize L
    P=np.identity(n) #initialize P
    for i in range(n): #parameterize pivots
        if U[i][i]==0:  #if zero where pivot expected
            k=i+1       #initialize k
            swapped=False #initialize swapped
            while k<n and swapped == False:
                if U[k][i] != 0:          #zero where pivot expected
                    (P,L,U)=swapper(P,L,U,i,k)
                    swapped=True          #terminate the while loop
                else:
                    k+=1
                    if k==n:
                        raise Exception("singular matrix") #this is optional, but might as well make it a bit more input-safe
        for k in range(i+1,n):  # we may now safely assume U[i][i]=0
            c=U[k][i] / U[i][i]   #compute which multiple of U[i] we'll need to subtract 
            U[k]=U[k] - c * U[i]  #clear out U[k][i]
            L=np.dot(L, elem_matrix(n,k,i,c)) #Right-multiply L by the appropriate elementary matrix
    return (P,L,U)
def is_pos_def(A,tolerance=10**(-10)):
    n=A.shape[0]
    sym_table= np.array([[abs(A[i,j]-A[j,i])<tolerance for i in range(n)] for j in range(n)]) #matrix of booleans telling us if A fails to be symmetric
    sym=np.all(sym_table)
    (P,L,U)=my_PLU(A)
    reg=(np.array_equal(P,np.identity(n))) #check P is trivial--if you implemented any pivoting, this might present an issue
    pos_piv_table=np.array([U[i,i]>tolerance for i in range(n)])
    pos_piv=np.all(pos_piv_table)
    return sym and reg and pos_piv

In [3]:
#testing
B=np.array([[1,2,3,4],[4,5,6,7],[-1,2,-3,4]],"float64")
K1=np.dot(B,np.transpose(B))
K2=np.dot(np.transpose(B),B)
K1, is_pos_def(K1),K2,is_pos_def(K2)
#What should be the correct output? Note rank(B)=3...

(array([[ 30.,  60.,  10.],
        [ 60., 126.,  16.],
        [ 10.,  16.,  30.]]),
 True,
 array([[18., 20., 30., 28.],
        [20., 33., 30., 51.],
        [30., 30., 54., 42.],
        [28., 51., 42., 81.]]),
 False)

In [4]:
#Testing pt 2
L=np.array([[1,1,1,1.],
[1,-1,1.,-1],
[1,1,-1.,-1],
[-1,1,1.,-1]],"float64")/2
D=np.diag([10**-2,10**-4,10**-10,10**-12])
K3= np.dot(L,np.dot(D,np.transpose(L)))
K3,is_pos_def(K3),is_pos_def(K3,10**-13)
# desired output: 
#(array([[ 0.002525,  0.002475,  0.002525, -0.002475],
#        [ 0.002475,  0.002525,  0.002475, -0.002525],
#        [ 0.002525,  0.002475,  0.002525, -0.002475],
#        [-0.002475, -0.002525, -0.002475,  0.002525]]),
# False,
# True)
# why does this make sense?

(array([[ 0.002525,  0.002475,  0.002525, -0.002475],
        [ 0.002475,  0.002525,  0.002475, -0.002525],
        [ 0.002525,  0.002475,  0.002525, -0.002475],
        [-0.002475, -0.002525, -0.002475,  0.002525]]),
 False,
 True)

### Minimization Problems
In section 5.2 of Olver-Shakiban, we learned a systematic approach for minimizing quadratic functions.
These are functions of the form $p(\vec x) = \vec x^T K \vec x -2\vec x^T \vec f +c$, where $K$ is a symmetric $n\times n$ matrix.
Another theorem gives the minimizer and minimum:

**Theorem 5.2.** If $K$ is a positive definite (and hence symmetric) matrix, then the quadratic function above has a unique minimizer, namely the solution $\vec x^*$ to the linear system $K \vec x =\vec f$. The minimum value $p(\vec x^*)$ is equal to $c-\vec f ^t \vec x^*$.

**(b)** Below, give a function which returns the minimizer and minimum of the quadratic function represented by the triple `K,f,c`.
Use a solving function from a previous activity to solve any linear systems (and not a built-in function).
Your function should `raise` an `Exception` if the supplied matrix fails to be positive definite (this will end up being important later)

**(c)** Use this to solve exercise 5.2.1: Find the global minimum value and global minimizer of the function $f(x,y,z)=x^2+2xy+3y^2+2yz+z^2-2x+3z+2$. Remember that any new arrays that you instantiate should be `"float64"` arrays.

In [5]:
def forward_sub(L,b): #input: lower-triangular matrix L, vector of compatible dimension b. 
    n=b.shape[0]
    c=np.zeros(n) #initialize output c
    for i in range(n):
        c[i]=(b[i]-sum([L[i][j]*c[j] for j in range(i)]))/L[i][i] #forward-substitution formula
    return c
def back_sub(U,c): #input: upper-triangular matrix L, vector of compatible dimension b. 
    n=c.shape[0]
    x=np.zeros(n) #initialize output x
    for i in range(n):
        x[-(1+i)]=(c[-(1+i)]-sum([U[-(i+1)][-(j+1)]*x[-(j+1)] for j in range(i)]))/U[-(i+1)][-(i+1)] #back-substitution formula.
    return x 
def my_solver(A,b):
    (P,L,U)=my_PLU(A)
    Pb=np.dot(P,b)
    c=forward_sub(L,Pb)
    x=back_sub(U,c)
    return x
def quadratic_min(K,f,c):
    if not is_pos_def(K):
        raise Exception("K fails to be positive definite")
    x_min=my_solver(K,f)
    p_min=c-np.dot(f,x_min)
    return (x_min,p_min)

In [6]:
#Testing
K=np.array([
    [1,1,1/2],
    [1,2,1/2],
    [1/2,1/2,2]
],"float64")
f=np.array([0,-3,7/2],"float64")
c=5
quadratic_min(K,f,c)
#desired output (array([ 2., -3.,  2.]), -11.0)

(array([ 2., -3.,  2.]), -11.0)

In [7]:
#part (c)
K=np.array([[1,1,0],[1,3,1],[0,1,1]],"float64")
f= np.array([-2,0,3],"float64")
c=2
quadratic_min(K,f,c)
#Answer: -12.0 at (-1,-1,4)

(array([-1., -1.,  4.]), -12.0)

## Exercise 2: Closest Point & Least Squares
### Closest Point
In the closest point problem, we are given a subspace $W\subset \mathbb{R}^m$ (typically with a basis given, say $\vec w_1,\dots \vec w_n$) and a vector $\vec b$ and asked to find the vector $\vec w \in W$ such that the distance $\left \lVert\vec w-\vec b \right \rVert$ is minimized.

Since minimizing the square of a positive quantity is equivalent to minimizing the quantity itself, this is equivalent to minimizing $\left \lVert\vec w-\vec b \right \rVert^2=\left \langle \vec w-\vec b,\vec w-\vec b\right \rangle=\left\langle \vec w,\vec w\right \rangle -2\left\langle \vec w,\vec b\right \rangle+\left \lVert \vec b\right \rVert^2$.
Writing $\vec w= x_1\vec w_1 +\dots +x_n\vec w_n$ then gives the expression $\left \lVert \vec w \right \rVert^2= \sum_{i,j=1}^n k_{ij}x_ix_j=\vec x^T K \vec x$ where $K=(k_{ij})$ is the Gram matrix of the basis $\vec w_1,\dots, \vec w_n$.
We also note that the same trick of writing out $\vec w$ in terms of the $\vec w_i$ yields an expression $\langle \vec w, \vec b \rangle=\sum x_i \langle \vec w_i, \vec b\rangle=\vec x^T \vec f$ where $f_i=\langle \vec w_i, \vec b\rangle$. In other words, we've expressed $\left \lVert\vec w-\vec b \right \rVert^2$ as the quadratic function $p(\vec x) = \vec x^T K \vec x -2\vec x^T \vec f +c$ where $K$ and $\vec f$ are given above and $c=\left \lVert \vec b \right \rVert^2$. 
The minimizer $\vec x^*$ to $p$ then gives the coefficients $x_1,\dots,x_n$ for the closest point $\vec w^*=x_1 \vec w_1+\dots +x_n \vec w_n$.

**Exercise (a):** Write a function which takes as input a matrix `W` whose columns are the vectors $\vec w_1,\dots \vec w_n$ and a vector `b` and returns the closest point $\vec w^*$ and its distance $\left \lVert \vec w^* -\vec b \right \rVert$. 
Do so by computing the appropriate objects `K`, `f` and `c`, passing those to `quadratic_min`, and transforming the output into appropriate form (for part of that, note that `np.sqrt` might be helpful).


In [8]:
def Gram(A): #this is a common enough need that we might as well make it its own function
    return np.dot(np.transpose(A),A)
def closest_point(W,b):
    K=Gram(W)
    f=np.dot(b,W)
    c=np.dot(b,b)
    (x_min,d_squared)=quadratic_min(K,f,c)
    w_min=np.dot(W,x_min)
    return w_min,np.sqrt(d_squared)
    

In [9]:
#Testing
W=np.transpose(np.array([
    [1,2,-1],
    [2,-3,-1]
    ],"float64"))
b=np.array([1,0,0],"float64")
closest_point(W,b)
#desired output:
#(array([ 0.66666667, -0.06666667, -0.46666667]), 0.5773502691896257)

(array([ 0.66666667, -0.06666667, -0.46666667]), 0.5773502691896257)

### Least Squares
The least squares problem for the system $A\vec x=\vec b$ is that of finding the $\vec x^*$ which minimizes the difference between the two sides. 
Olver-Shakiban gives a theorem which computes $\vec x^*$ directly:

**Theorem 5.11.** Assume $\ker A = \{\vec 0 \}$. Then, the least squares solution to $A\vec x = \vec b$ under the Euclidean norm is $\vec x^*$, the unique solution to the *normal equations*: $(A^T A) \vec x = A^T \vec b$.
The *least squares error* is $\left \lVert A \vec x^* - \vec b\right \rVert^2$.

**Exercise (b):** Modify your function from part (a) to give a function `least_squares` which provides the (more general) least squares solution and least squares error for a given matrix `A` and vector `b`. 

*Hint:* This should be a matter more-or-less of *removing* code from the previous function

In [10]:
def least_squares(A,b):
    K=Gram(A)
    f=np.dot(b,A)
    c=np.dot(b,b)
    (x_min,d_min)=quadratic_min(K,f,c)
    return x_min,d_min
    

In [11]:
#Testing:
A=np.array([
    [1,2,0],
    [3,-1,1],
    [-1,2,1],
    [1,-1,-2],
    [2,1,-1]
    ],"float64")
b=np.array([1,0,-1,2,2],"float64")
least_squares(A,b)
#Desired Output: (array([ 0.4118705 ,  0.24820144, -0.95323741]), 0.0323741007194247)

(array([ 0.4118705 ,  0.24820144, -0.95323741]), 0.0323741007194247)

## Exercise 3: Data-fitting
Read through the beginning of section 5.5 on Data Fitting and Interpolation on pages 254-258 of Olver-Shakiban.

**Exercise: (a)** Write a function `linear_fit` which takes as input two vectors `t` and `y`. Use `least_squares` to compute $\alpha,\beta$ and print "Line of best fit is y=$\alpha$+$\beta$t", then return $(\alpha,\beta)$. 

(So for instance, if you find $\alpha=3.1415$ and $\beta=2.7183$, your function should print "Line of best fit is y= 3.1415 + 2.7183 t").

Then, use `linear_fit` to solve problem 5.5.3(b) in Olver-Shakiban (estimating the price of a home in 2005 and 2010). You may [optionally] want to do this by writing another function that wraps `linear_fit` by taking an additional parameter $t_0$ (or parameters) and returning the predicted value $y(t_0)$ in service of your answer

In [12]:
def linear_fit(t,y):
    A=np.array([[1., ti] for ti in t])
    v=least_squares(A,y)[0]
    print("line of best fit is y=",v[0]," + ",v[1],"t")  
    return (v[0],v[1])
    
    

In [13]:
#Testing
t=np.array([0,1,3,6.])
y=np.array([2,4,7,12.])
linear_fit(t,y)
#desired output:
#line of best fit is y= 2.1428571428571432  +  1.6428571428571428 t
#(2.1428571428571432, 1.6428571428571428)

line of best fit is y= 2.1428571428571432  +  1.6428571428571428 t


(2.1428571428571432, 1.6428571428571428)

In [14]:
#Problem 5.5.3
# year:   1989   1990   1991   1992   1993   1994   1995   1996   1997   1998   1999
# amount: 86.4   89.8   92.8   96.0   99.6   103.1  106.3  109.5  113.3  120.0  129.5
#optional helper function:
def linear_predict(t,y,t_0):
    a,b=linear_fit(t,y)
    return a +b*t_0
#(or even)
def linear_predict2(t,y,*t_m):
    a,b=linear_fit(t,y)
    return [(t_0,a +b*t_0) for t_0 in t_m]
t=np.array([1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999],"float64")
y=np.array([86.4,89.8,92.8,96.0,99.6,103.1,106.3,109.5,113.3,120.0,129.5],"float64")

p_2005=linear_predict(t,y,2005)
p_2010=linear_predict(t,y,2010)
print(p_2005,p_2010)

#or
linear_predict2(t,y,2005,2010)

#answer is: in 2005, 147.3590909091381; in 2010, 166.97272727279596

line of best fit is y= -7717.709090917531  +  3.922727272731506 t
line of best fit is y= -7717.709090917531  +  3.922727272731506 t
147.3590909091381 166.97272727279596
line of best fit is y= -7717.709090917531  +  3.922727272731506 t


[(2005, 147.3590909091381), (2010, 166.97272727279596)]

**(b):**
Now, read the second portion of section 5.5 on pages 259-260 through theorem 5.60.
Write a function `poly_fit` which implements the process for finding a line of best polynomial fit for a set of $t$-values `t` and a set of $y$-values `y`.
Your function should take as an additional input an integer `n` and a parameter `t_0`.
Return the result of inputting `t_0` to the polynomial giving the degree-`n` line of best fit.
Do this using least squares by implementing the Vandermonde matrix presented in the text (probably as its own function).

*Challenge (optional):* include a line which prints the formula for the line of best fit in human-readable form. 

In [15]:
def general_vandermonde(n,t=[]):
    m=len(t)
    return np.array([[t_i**j for j in range(n+1)] for t_i in t],"float64")
def poly_string(x):
    poly="line of best fit is "
    deg=0-6.615226941131597
    for a in x:
        poly=poly+"("+str(a)+")*t^"+str(deg)+" +"
        deg+=1
    poly=poly[:-1]
    return poly
def poly_fit(n,t,y,t_0):
    A=general_vandermonde(n,t)
    x=least_squares(A,y)[0]
    print(poly_string(x))
    deg =0
    tot=0
    for a in x:
        tot=tot+a * t_0**deg
        deg+=1
    return tot


In [16]:
#testing
t=np.array([1,2,3,4],"float64")
y=np.array([-1,2,-3,4],"float64")
poly_fit(2,t,y,2.5)
# desired output:
# -0.75

line of best fit is (3.0)*t^-6.615226941131597 +(-4.0)*t^-5.615226941131597 +(1.0)*t^-4.615226941131597 


-0.75

In [17]:
#testing part 2
#testing
t=np.array([1,10,10**2,10**3,10**4],"float64")
y=np.array([-1,1,-3,4,0],"float64")
poly_fit(3,t,y,10**(2.5))
# desired output:
# -6.615226941131597

line of best fit is (0.125325100672444)*t^-6.615226941131597 +(-0.034181836967189)*t^-5.615226941131597 +(4.1904007237925995e-05)*t^-4.615226941131597 +(-3.848707678130196e-09)*t^-3.6152269411315974 


-6.615226941131597

**(c):** Try the example below. What happens and why?

In [18]:
#testing
t=np.array([1,2,3,4],"float64")
y=np.array([-1,2,-3,4],"float64")
#poly_fit(4,t,y,np.sqrt(2))#UNCOMMENT ME!!!


**Exercise:** Correct this issue. Give a function `poly_fit_safe` which has the same behavior except avoiding this pitfall. Your function should always return an approximation of the greatest reasonable degree less than or equal to `n`.

<details>
    <summary>
        <b> Hint:</b> (Click here to open)
    </summary>
    
- The issue is that there are *many* possible lines of best fit when we ask for one of too high degree. Correct this issue by simply changing the degree to the greatest reasonable choice when that happens.
    
</details>

In [19]:
def poly_fit_safe(n,t,y,t_0):
    n_max=t.shape[0]-1
    if n>n_max:
        m=n_max
        print("input degree too high, defaulting to n=", m)
    else:
        m=n
    A=general_vandermonde(m,t)
    x=least_squares(A,y)[0]
    print(poly_string(x))
    deg =0
    tot=0
    for a in x:
        tot=tot+a * t_0**deg
        deg+=1
    return tot


In [20]:
#testing
t=np.array([1,2,3,4],"float64")
y=np.array([-1,2,-3,4],"float64")
poly_fit_safe(4,t,y,np.sqrt(2))
#desired output: 2.495791138430997

input degree too high, defaulting to n= 3
line of best fit is (-32.000000000005315)*t^-6.615226941131597 +(51.6666666666751)*t^-5.615226941131597 +(-24.000000000003787)*t^-4.615226941131597 +(3.3333333333338384)*t^-3.6152269411315974 


2.495791138430997